In [179]:
# manipulation data
import pandas as pd
import numpy as np

#visualiation data
import matplotlib.pyplot as plt
import seaborn as sns 
import matplotlib
import plotly.graph_objects as go
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot


#from dataprep.eda import *
#from dataprep.datasets import load_dataset
#from dataprep.eda import create_report

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import r2_score, classification_report
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier



from sklearn.model_selection import GridSearchCV
from sklearn.utils.class_weight import compute_class_weight

import math
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder



from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression



import datetime as dt
import requests
import json

from scipy import stats

from googleapiclient.discovery import build
import praw


In [249]:
reddit = praw.Reddit(client_id ='<reddit_client_id>',
client_secret ='<reddit_secret>',
user_agent ='sentiment analysis')

print(reddit.read_only)


True


In [250]:
#Reddit post - https://www.reddit.com/r/kitchener/comments/18dc0ov/positive_step_towards_our_society/
submission = reddit.submission(id='18dc0ov')


In [182]:
print(f"Post Title: {submission.title}\n")


Post Title: Positive step towards our society



In [183]:
with open(output_file_path, "w", encoding="utf-8") as output_file:
    for comment in submission.comments.list():
        output_file.write(f"Comment: {comment.body}\n")

In [248]:

api_key = '<google_api_key>'
youtube = build('youtube', 'v3', developerKey=api_key)

#YouTube video - https://www.youtube.com/watch?v=8_RL4Lq-OG0
video_id = '8_RL4Lq-OG0'

top_level_request = youtube.commentThreads().list(
    part='snippet',
    videoId=video_id,
    textFormat='plainText',
)

top_level_response = top_level_request.execute()
comments = []
for top_level_comment in top_level_response['items']:
    author = top_level_comment['snippet']['topLevelComment']['snippet']['authorDisplayName']
    text = top_level_comment['snippet']['topLevelComment']['snippet']['textDisplay']
    comments.append(f"Comment: {text}\n")


HttpError: <HttpError 400 when requesting https://youtube.googleapis.com/youtube/v3/commentThreads?part=snippet&videoId=8_RL4Lq-OG0&textFormat=plainText&key=%3Cgoogle_api_key%3E&alt=json returned "API key not valid. Please pass a valid API key.". Details: "[{'message': 'API key not valid. Please pass a valid API key.', 'domain': 'global', 'reason': 'badRequest'}]">

In [185]:
with open(output_file_path, 'a', encoding='utf-8') as file:
    for comment in comments:
        file.write(comment)

In [186]:
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

input_file_path = "../data/text_based_posts.txt"

with open(input_file_path, "r", encoding="utf-8") as input_file:
    examples = input_file.read().split("Comment:")

annotated_dataset = []

for example in examples:
    if not example.strip():
        continue
    
    text_content = example.strip()
    
    sentiment_scores = sia.polarity_scores(text_content)
    
    if sentiment_scores['compound'] >= 0.05:
        sentiment_label = 'positive'
    elif sentiment_scores['compound'] <= -0.05:
        sentiment_label = 'negative'
    else:
        sentiment_label = 'neutral'
    
    annotated_dataset.append({
        'text_content': text_content,
        'sentiment': sentiment_label,
        'sentiment_score': sentiment_scores['compound']
    })


In [187]:
len(annotated_dataset)

91

In [188]:
len(examples)

92

In [189]:
annotated_dataset[0]

{'text_content': "It's a nice start.\n\nThey need to introduce random checks to insure that money is still in the account and being used by the student after they are accepted into Canada to study as well.",
 'sentiment': 'positive',
 'sentiment_score': 0.7184}

In [190]:
neutral_count = 0
pos_count = 0
neg_count = 0
for entry in annotated_dataset:
    if entry['sentiment'] == 'neutral':
        neutral_count += 1
    if entry['sentiment'] == 'negative':
        neg_count += 1
    if entry['sentiment'] == 'positive':
        pos_count += 1
    
print("Length of Neutral Sentiment Tags:", neutral_count)
print("Length of Positive Sentiment Tags:", pos_count)
print("Length of Negative Sentiment Tags:", neg_count)

Length of Neutral Sentiment Tags: 30
Length of Positive Sentiment Tags: 44
Length of Negative Sentiment Tags: 17


In [191]:
df = pd.DataFrame(annotated_dataset)
df

,text_content,sentiment,sentiment_score
0,It's a nice start.\n\nThey need to introduce r...,positive,0.7184
1,Now time to crack down on these Dollarama coll...,negative,-0.3818
2,So the organizations these students use to spo...,negative,-0.1477
3,It is. Praying for them to raise it to 55k tho...,positive,0.3612
4,"CBC article on it,\n\n[https://www.cbc.ca/news...",neutral,0.0000
...,...,...,...
86,Next time visit Crafty Ramen! Its just down th...,neutral,0.0000
87,First tip of enjoying kitchener: Get outta dow...,positive,0.7339
88,I am 16 turning 17 and I have been here all my...,negative,-0.4562
89,Being a guy who walks through all of the first...,neutral,0.0000


In [192]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer, ENGLISH_STOP_WORDS
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import demoji
from gensim.models import Word2Vec


In [193]:
demoji.download_codes()

C:\Users\Rj\AppData\Local\Temp\ipykernel_23644\2299328559.py:1: FutureWarning:

The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.



In [194]:
stop_words_list = list(ENGLISH_STOP_WORDS)


In [195]:
X = df['text_content']
y = df['sentiment']
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [196]:
# Method 1: Count Vectorizer
count_vectorizer = CountVectorizer(stop_words=stop_words_list)
X_train_count = count_vectorizer.fit_transform(X_train)
X_test_count = count_vectorizer.transform(X_test)

In [197]:
# Method 2: TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words_list)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [202]:
# Method 3: Hashing Vectorizer
hashing_vectorizer = HashingVectorizer(stop_words=stop_words_list, n_features=5000, alternate_sign=False)
X_train_hashing = hashing_vectorizer.transform(X_train)
X_test_hashing = hashing_vectorizer.transform(X_test)

In [198]:
# download word2vec - https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit

In [199]:
word2vec_model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)


In [200]:
def doc_vectorizer_w2v(doc, model, num_features):
    words = doc.split()
    feature_vector = np.zeros((num_features,), dtype="float32")
    num_words = 0
    for word in words:
        if word in model:
            num_words += 1
            feature_vector = np.add(feature_vector, model[word])
    if num_words > 0:
        feature_vector = np.divide(feature_vector, num_words)
    return feature_vector

In [201]:
X_train_w2v = np.array([doc_vectorizer_w2v(doc, word2vec_model, 300) for doc in X_train])
X_test_w2v = np.array([doc_vectorizer_w2v(doc, word2vec_model, 300) for doc in X_test])


In [216]:
models = {
    "XGBClassifier": XGBClassifier(),
    "SVC":SVC(),
    "LogisticRegression": LogisticRegression(),
    "RandomForestClassifier": RandomForestClassifier(),
    "MultinomialNB": MultinomialNB()
}

In [217]:
# Dictionary to store X_test variables
X_tests = {
    "count": X_test_count,
    "tfidf": X_test_tfidf,
    "hashing": X_test_hashing,

}

In [218]:
best_accuracy = 0
best_model = None
best_method = None

In [219]:
for model_name, model in models.items():
    for method, X_test_method in X_tests.items():
        train_variable_name = f'X_train_{method.replace(" ", "_").lower()}'
        if train_variable_name not in globals():
            print(f"Warning: {train_variable_name} not found in globals(). Skipping.")
            continue
        model.fit(globals()[train_variable_name], y_train)
        y_pred = model.predict(X_test_method)
        accuracy = accuracy_score(y_test, y_pred)
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_model = model_name
            best_method = method


In [220]:
best_accuracy

0.47368421052631576

In [221]:
best_model

'SVC'

In [222]:
best_method

'count'

In [226]:
mnb_model = SVC()

In [227]:
mnb_model.fit(X_train_count,y_train)

SVC()

In [228]:
y_pred = mnb_model.predict(X_test_count)


In [229]:
acc = accuracy_score(y_test, y_pred)
class_rep = classification_report(y_test, y_pred)

E:\Conestoga\ML programming\Sentiment Analysis\venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

E:\Conestoga\ML programming\Sentiment Analysis\venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

E:\Conestoga\ML programming\Sentiment Analysis\venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [230]:
acc

0.47368421052631576

In [231]:
print(class_rep)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       0.60      0.50      0.55         6
           2       0.43      0.75      0.55         8

    accuracy                           0.47        19
   macro avg       0.34      0.42      0.36        19
weighted avg       0.37      0.47      0.40        19



In [244]:
param_grid = {'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001], 'kernel': ['linear','rbf']}
svc_model = SVC(random_state=42)
grid_search = GridSearchCV(svc_model, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train_count,y_train)
best_svc_model = grid_search.best_estimator_

In [245]:
best_svc_model

SVC(C=10, gamma=0.1, random_state=42)

In [246]:
best_svc_model = grid_search.best_estimator_
y_pred = best_svc_model.predict(X_test_count)

# Evaluate the model
acc = accuracy_score(y_test, y_pred)
class_rep = classification_report(y_test, y_pred)

In [247]:
acc

0.42105263157894735